# Agnostic Simulator: QPU connection

This notebook elaborates on how `agnostic_simulator` can facilitate hardware experiments by directly integrating the API provided by some hardware providers (Honeywell, IonQ...) or broader quantum cloud services such as Azure Quantum or Braket.


## Table of contents
* [1. IonQ Rest API](#1)
    * [1.1 References](#1.1)
    * [1.2 Environment](#1.2)
    * [1.3 Quantum Circuit](#1.3)
    * [1.4 IonQConnection class](#1.4)
* [2. Honeywell Rest AI](#2)
    * [2.1 References](#2.1)
    * [2.2 Environment](#2.2)
    * [2.3 Quantum Circuit](#2.3)
    * [2.4 HoneywellConnection class](#2.4)
* [3. Azure Quantum](#3)
* [4. Braket](#4)

---


## Requirements

In order to run the contents of this notebook, I recommend that you first install the agnostic simulator package as per the instructions, relying on the `setup.py` file in the github repository.

Ensure you have valid ID tokens and logins for the services of your choice (IonQ, Honeywell, ...). This may require 1QBit to request them from the hardware providers.

---

## 1. IonQ Rest API <a class="anchor" id="1"></a>


### 1.1 References <a class="anchor" id="1.1"></a>

We merely provide a few Python wrappers around the REST API provided by IonQ, to facilitate job submission and monitoring. The most up-to-date list of supported features and information regarding the IonQ API is available below (some special privileges may be required to access this material).

https://dewdrop.ionq.co/

https://docs.ionq.co

### 1.2 Environment <a class="anchor" id="1.2"></a>

Users allowed to use the IonQ REST services should have been provided with an ID token (a rather long string of alphanumeric characters and dashes). For the REST requests to be succesful, users are expected to set their `IONQ_APIKEY` environment variable to this value.

This can be done in your bash terminal (`export IONQ_APIKEY=<value>`), or in your Python script (`os.environ['IONQ_APIKEY'] = <value>`). Please make sure you do not upload or share code showing sensitive information, such as logins / tokens. If you export these variables before launching the Jupyter notebook server, they will be visible to the notebook.

### 1.3 Quantum circuit <a class="anchor" id="1.3"></a>

IonQ relies on a general JSON representation for quantum circuits, which `agnostic_simulator` can generate from an abstract circuit, using the `translator` module. IonQ also offer partial support for other common formats (OpenQASM, ...). Formats and quantum operations supported are available in their documentation. We chose to rely on the best-supported format, their JSON format, which takes the shape of nested list and dictionaries.

Below, we show how users can produce a simple quantum circuit in this format:

In [ ]:
from agnostic_simulator.translator import Circuit, Gate, translate_json_ionq

circ1 = Circuit([Gate("H", 0), Gate("X", 1)])
json_circ1 = translate_json_ionq(circ1)
print(json_circ1)

### 1.4 IonQConnection class <a class="anchor" id="1.4"></a>

The `IonQConnection` class encapsulates a collection of wrappers to the IonQ REST API. Internally, it stores information about the endpoint and the authorization header, containing your identification token. This class only is instantiated succesfully if your ID token has been set properly.

More generally speaking, all calls to the REST API are checked for errors, and would return the IonQ error message corresponding to the unsuccesful request.

In [ ]:
from agnostic_simulator.qpu_connection import IonQConnection

# import os
# os.environ["IONQ_APIKEY"] = <value>

ionq_connection = IonQConnection()

#### job submission

Job submission can be achieved through the `job_submit` method, which attempts to submit a job on a simulator or QPU, and returns a job ID if submission was succesful.

This method takes input arguments that need to be provided by the user:

- the target backend (currently, IonQ provides two options: 'simulator' and 'qpu')
- the quantum circuit (assumed to be in JSON format, by default)
- the number of shots required (ignored by the simulator)
- a name for your job
- any other option as key arguments (see source code and IonQ documentation)

Assuming a valid API key, and no issue with the IonQ services themselves, we can then submit a simple job targeting their statevector simulator. The status of the job may be in various states (queued, ready, running ...) depending on the status of the IonQ services themselves.

In [ ]:
job_id = ionq_connection.job_submit('simulator', json_circ1, 100, 'test_simulator_json_job')

#### job history and job info

As of July 2020, IonQ does not provide an online portal or dashboard allowing users to monitor their jobs. Users can however access their job history and info through the `job_get_history` and `job_get_info` methods, shown as below.

Depending on the timing of your REST requests, the job info may differ widely, from a failed job to a completed job with results included.

In [ ]:
job_info = ionq_connection.job_get_info(job_id)
print(job_info)

The output of `job_get_history` should at least feature the job we just submitted, and can also show up to the last 25 submitted jobs that were not deleted from IonQ's database. The output is a `pandas` dataframe, in order to facilitate parsing of information.

In [ ]:
job_history = ionq_connection.job_get_history()
print(job_history)

#### job results

The method `job_get_results` provides a wrapper to a blocking call, querying for the state of the target job at regular intervals, attempting to retrieve the results. 

If the job has successfully completed, the 'data' entry of the json return dictionary is available to the user. Running a job using the `simulator` backend yields a histogram of probabilities. When running on a QPU, the data may be different, a may take the form of a list of all the shots, or a histogram agglomerating the counts into bins.

IonQ raw results use a "most-significant qubit first" representation, encoded as an integer. It means that if the integer `i` is our result dictionary, the binary representation of `i`, which is a string of zeros and ones, correspond to $q_0 q_1...q_n$ (e.g measured qubit basis states are to be read left-to-right). This convention usually differs from the one used in theoretical literature, which usually reads right-to-left.

For example,for a 2-qubit circuit:

- 0 is '00' in binary, indicating $q_0$ mesured in state $|0\rangle$, and $q_1$ in state $|0\rangle$
- 1 is '01' in binary, indicating $q_0$ mesured in state $|1\rangle$, and $q_1$ in state $|0\rangle$
- 2 is '10' in binary, indicating $q_0$ mesured in state $|0\rangle$, and $q_1$ in state $|1\rangle$
- 3 is '11' in binary, indicating $q_0$ mesured in state $|1\rangle$, and $q_1$ in state $|1\rangle$

In the future, `agnostic_simulator` may return these in its own format, following a given convention.

In [ ]:
results = ionq_connection.job_get_results(job_id)
print(results)

#### job cancel / delete

A wrapper called `job_cancel` provides a method to cancel before execution (if the job hasn't run yet), or delete a job from the history. The cell below cancel / delete the previously submitted job: it therefore should not appear in the history anymore.

In [ ]:
ionq_connection.job_cancel(job_id)
job_history = ionq_connection.job_get_history()
print(f"\n{job_history}")

## 2. Honeywell Rest API<a class="anchor" id="2"></a>


### 2.1 References <a class="anchor" id="2.1"></a>

We merely provide a few Python wrappers around the REST API provided by Honeywell, to facilitate job submission and monitoring. The most up-to-date list of supported features, error codes and information regarding their API is available below (some special privileges may be required to access this material).

https://drive.google.com/file/d/1PDQwIyAVlMn5JEMVWBKvEPkcawzh8EeG/view?usp=sharing

Honeywell provides a portal / dashboard to facilitate job monitoring. Your 1QBit admin (currently Valentin Senicourt) can create accounts for new users:

https://um.qapi.honeywell.com/index.html

As of July 2020, this dashboard is fairly simple and looks like this:

![Honeywell dashboard](img/honeywell_dashboard.png)


### 2.2 Environment <a class="anchor" id="2.2"></a>

For the REST requests to be succesful, users are expected to set their `HONEYWELL_EMAIL` and `HONEYWELL_PASSWORD` environment variables to the correct values, which are the logins used for the online portal above.

This can be done in your bash terminal (`export HONEYWELL_EMAIL=<value>`), or in your Python script (`os.environ['HONEYWELL_EMAIL'] = <value>`). Please make sure you do not upload or share code showing sensitive information, such as logins / tokens. If you export these variables before launching the Jupyter notebook server, they will be visible to the notebook.

### 2.3 Quantum circuit <a class="anchor" id="2.3"></a>

Honeywell supports the OpenQASM 2.0 format. Users can use the `translate_openqasm` function from the `translator` module in `agnostic_simulator` to obtain their circuit in an acceptable format. Otherwise, many packages (Qiskit, etc) can also easily export quantum circuits in this format as well, in case `agnostic_simulator` does not support all the relevant instructions.

In [ ]:
from agnostic_simulator.translator import Circuit, Gate, translate_openqasm

circ1 = Circuit([Gate("H", 0), Gate("X", 1)])
qasm_circ1 = translate_openqasm(circ1)
print(qasm_circ1)

### 2.4 HoneywellConnection class <a class="anchor" id="2.4"></a>

The `HoneywellConnection` class encapsulates a collection of wrappers to their REST API. Internally, it stores information about the endpoint and the authorization header, containing your identification token. This class only is instantiated succesfully if your login information has been set properly.

More generally speaking, all calls to the REST API are checked for errors, and would return the REST error message corresponding to the unsuccesful request.

In [ ]:
from agnostic_simulator.qpu_connection import HoneywellConnection

# import os
# os.environ["HONEYWELL_EMAIL"] = <value>
# os.environ["HONEYWELL_PASSWORD"] = <value>

honeywell_connection = HoneywellConnection()

#### available devices

Honeywell enables users to query for the available devices. Not all of them are QPUs: some devices have names finishing in 'APIVAL' (API validation) and can be targeted for the purpose of validiation or verification before attempting to submit a job to an actual QPU.

We return a dictionary allowing users to see information about each available device:

In [ ]:
available_devices = honeywell_connection.get_devices()
print(available_devices)

#### job submission

Job submission can be achieved through the `job_submit` method, which attempts to submit a job to a device, and returns a job ID if submission was succesful.

This method takes input arguments that need to be provided by the user:

- the target device (presumably appearing in the list of available devices)
- the quantum circuit (assumed to be in OpenQASM 2.0 format, by default)
- the number of shots required
- a name for your job
- any other option as key arguments (see source code and Honeywell documentation)

Assuming a valid API key, and no issue with the services themselves, we can then submit a simple job targeting their API validation device. This job should finish almost instantly, as it performs no simulation: the validation device simply pretends all qubits were measured in state $|0\rangle$.

In [ ]:
n_shots = 10
target_device = 'HQS-LT-1.0-APIVAL'

job_id = honeywell_connection.job_submit(target_device, qasm_circ1, n_shots, '1qbit_test_submit_job')

#### job history and job info

Honeywell portal / dashboard is probably the most convenient tool to know the current status of your jobs (see image in `Reference` section above).

Depending on the timing of your REST requests, the job info may differ widely, from a failed job to a completed job with raw results included.

In [ ]:
job_info = honeywell_connection.job_get_info(job_id)
print(f"\n{job_info}")

#### job results

The method `job_get_results` provides a wrapper to a blocking call, querying for the state of the target job at regular intervals, attempting to retrieve the results. 

If the job has successfully completed, the results are returned. Raw results show the outcome of each of the shots, in order. Later on, we may provide an option to return these as a sparse histogram with a given convention for qubit order.

For any question regarding ordering of the qubits in the output, please refer to the Honeywell documentation.
In an APIVAL device has been selected, no simulation occurs and the result assume an all-zero state for each shot.

In [ ]:
results = honeywell_connection.job_get_results(job_id)
print(results)

#### job cancel

Assuming the job has not run already, it is possible to cancel it using the job ID. In the event that it is not possible to cancel the job, the `job_cancel` method simply prints and returns the error message and status of the target job to provide the user with as much information as possible.

In [ ]:
job_status = honeywell_connection.job_cancel(job_id)
print(job_status)

## 3. Azure Quantum <a class="anchor" id="3"></a>

Though this package does not currently provide a way to directly submit jobs through Microsoft's Azure Quantum cloud services, the `translate_qsharp` function in the `translator` module can parse an abstract circuit and generate Q# code that can be written to file.

This code is compatible with both the local QDK simulator (good for testing before submitting to an actual QPU) or by Azure Quantum. Submission through Azure Quantum will require the user to have an account on Azure, install the local CLI and Python packages if needed.

For an example of how one can use this package to first generate circuits, and then submit jobs through Azure quantum, please look into the `example/qsharp` folder of this package.

If you are a 1QBit employee, the following Confluence page may contain useful information regarding the use of Azure Quantum: https://1qbit-intra.atlassian.net/wiki/spaces/QSD/pages/1319600305/Submit+monitor+Hardware+Experiments+on+Azure+Quantum

## 4. Braket <a class="anchor" id="3"></a>

This package currently offers to generate a quantum circuit in the Braket format using the `translate_braket` function from the translator module. Users can then use the Braket python SDK provided by Amazon (`amazon-braket-sdk` to submit experiments to backends available in the cloud and retrieve results, or simply run them using the [LocalSimulator](https://docs.aws.amazon.com/braket/latest/developerguide/braket-get-started-run-circuit.html)   that comes with the SDK.

The following relies on the AWS CLI, and assumes you have an IAM user with proper permissions (Braket, S3 buckets). You can also use the Braket services through your web browser, which gives you access to managed python notebooks. This may or may not suited to your needs: deviating from this environment (installing new python packages etc) or moving data from your personal computer to that environment may be annoying.


### 4.1 Installation tips for use of Braket through the CLI


a) [Install CLI](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html)
> aws --version

b) IAM user: [Create access key id & secret access key](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_access-keys.html#Using_CreateAccessKey), then [configure your CLI profile](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-profiles.html) with them
(plug the requested information as prompted by the command below, or just press enter for all and then directly edit `~/.aws/credentials`)

> aws configure 

c) Request from your admin proper permissions policies (Braket, S3 buckets), [enable Braket](https://docs.aws.amazon.com/braket/latest/developerguide/braket-enable-overview.html)

### 4.2 Submitting and retrieving results

Let's simply build a circuit preparing a Bell state, and show how to submit the experiment and retrieve the results using the `amazon-braket-sdk`. In the future, we may provide wrappers around their API in order to facilitate the process or ensure the data is in our standard format before post-processing. First, we use `agnostic_simulator` 's `translate_braket` function to produce a quantum circuit in the Braket format:

In [ ]:
from agnostic_simulator import Gate, Circuit
from agnostic_simulator.translator import translate_braket

abstract_circuit = Circuit([Gate('H',0), Gate('CNOT', target=1, control=0)])
braket_circuit = translate_braket(abstract_circuit)

print(braket_circuit)

Then, we use the `boto3` package, yfor your script to use your CLI credentials / account.
We need to provide the name of the S3 bucket that will be used to store your results: all experiments sent to Braket return us a result object for you to use in your script, but a duplicate is always saved on a S3 bucket for later use.

We elect to pick the sv1 simulator backend for this example, to ensure low cost and availability for this code cell. The results come back to us as a data structure that you are free to explore.

In [ ]:
# Get the account ID, path to s3 bucket used for results
import boto3
from braket.aws import AwsDevice
from braket.devices import LocalSimulator

aws_account_id = boto3.client("sts").get_caller_identity()["Account"]
s3_location = ("amazon-braket-76c619ed45ad", "bell-test")

# choose the cloud-based managed simulator to run your circuit
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
n_shots = 100
res = device.run(braket_circuit, s3_location, shots=n_shots).result()

counts = res.measurement_counts
print(counts)

The `measurement_counts` method returns an object of the built-in Python `Counter` class, which can then be passed to other functions: here we compute an expectation value.

In [ ]:
from agnostic_simulator import Simulator
from agnostic_simulator.helper_circuits import pauli_string_to_of

# Rescale for frequencies
freqs = {k:v/n_shots for k,v in freqs.items()}
print(f"Frequencies: {freqs}")

# Compute expectation value for ZZ operator
exp_ZZ = Simulator.get_expectation_value_from_frequencies_oneterm(pauli_string_to_of("ZZ"), freqs2)
print(f"Expectation value for ZZ operator {exp_ZZ}")